In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### 236 средние сроки по этапам

   ###### все договоры

In [2]:
# ввод названия выгруженного датасета
data_236 = input()
df = pd.read_excel(data_236)

july_236_2020.xls


In [3]:
df[(df['Step'] == 'Подтверждение заключенности') | (df['Step'] == 'Подтверждение заключенности договора регистратором')].nunique()['ID contract']

460

In [4]:
def missingdata(data):
    '''
    считаем кол-во и долю пропущеных значений в датасете
    '''
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    return ms

missingdata(df)

,Total,Percent
Department,1850,29.891743
ZNP,1232,19.906285
Description,661,10.680239
Department_1,26,0.420100
Direction,19,0.306996
Author,9,0.145419


In [5]:
# смотрим фио работников с незаполненной дирекцией
df[(df['Direction'].isnull())].groupby(['Author']).nunique()['ID contract']

Author
Киселев Владимир Геннадьевич    2
Name: ID contract, dtype: int64

In [6]:
def direction(d1):
    '''
    функция заполняющая дирекции, дополняется в ручную
    '''
    if d1 == 'Шупеник Алена Викторовна':
        return 'Дирекция по правовым и корпоративным вопросам'
    elif d1 == 'Киселев Владимир Геннадьевич':
        return 'Служба маркетинга'
    elif d1 == 'Молокотина Елена Викторовна':
        return 'Дирекция по недропользованию'
    else:
        return 0

# заполняем пропуски по дирекциям
df['Direction'].fillna(df['Author'].apply(direction), inplace = True)

In [7]:
# получаем датасет аггрегированный по этапу с суммой по дням выполнения и кол-вом этапов
df_all = df.groupby(['Step']).agg(['sum', 'count'])['Time_days'].reset_index()
# переименовываем столбец 
df_all.rename(columns={'count': 'count_step'}, inplace=True)
# получаем датасет сгруппированный по этапос с уникальным количеством договоров
df_all_contract = df.groupby(['Step']).nunique()['ID contract'].reset_index()
# переименовываем столбец
df_all_contract.rename(columns={'ID contract': 'count_contract'}, inplace=True)
# соединяем два датасета по названию этапа. теперь есть столбцы с кол-вом этапов, кол-вом договоров и суммой времени выполнения этапа
df_all = df_all.merge(df_all_contract, on = 'Step', how = 'left')
# считаем среднее время выполнения этапа по кол-ву договоров
df_all['Time_contract'] = df_all['sum']/df_all['count_contract']
# считаем среднее время выполнения этапа по кол-ву этапов
df_all['Time_step'] = df_all['sum']/df_all['count_step']
# считаем усредненное время выполнения этапа
df_all['Time_avg'] = (df_all['Time_contract'] + df_all['Time_step'])/2
# записываем датасет в эксель файл
df_all.to_excel('df_all.xlsx')

   ###### в разрезе дирекций

In [8]:
# создаем список дирекций для расчета среднего срока выполнения этапов
list_eng = ['Tech',
 'PD',
 'IT',
 'Legacy',
 'Mine',
 'HR',
 'CC']

In [9]:
def direction_eng(d2):
    '''
    функция соответсвия дирекций в списке и в датасете
    '''
    if d2 == 'Техническая дирекция':
        return 'Tech'
    elif d2 == 'Дирекция по закупкам':
        return 'PD'
    elif d2 == 'Дирекция по информационным технологиям':
        return 'IT'
    elif d2 == 'Дирекция по правовым и корпоративным вопросам':
        return 'Legacy'
    elif d2 == 'Дирекция по недропользованию':
        return 'Mine'
    elif d2 == 'Дирекция по персоналу':
        return 'HR'
    elif d2 == 'Дирекция по капитальному строительству':
        return 'CC'
    else:
        return 'other_direction'

In [10]:
# создаем столбец с названием дирекции из списка
df['Direction_new'] = df['Direction'].apply(direction_eng)

In [11]:
# объявляем глобальную переменную
glb = globals()
# в цикле создаем эксель файлы с расчитанными средними сроками выполнения каждого этапа для дирекций из списка
for i in list_eng:
    glb['df_'+i] = df[df['Direction_new'] == i].groupby(['Step']).agg(['sum', 'count'])['Time_days'].reset_index()
    glb['df_'+i].rename(columns={'count': 'count_step'}, inplace=True)
    glb['df_'+i+'_contract'] = df[df['Direction_new'] == i].groupby(['Step']).nunique()['ID contract'].reset_index()
    glb['df_'+i+'_contract'].rename(columns={'ID contract': 'count_contract'}, inplace=True)
    glb['df_'+i] = glb['df_'+i].merge(glb['df_'+i+'_contract'], on = 'Step', how = 'left')
    glb['df_'+i]['Time_contract'] = glb['df_'+i]['sum']/glb['df_'+i]['count_contract']
    glb['df_'+i]['Time_step'] = glb['df_'+i]['sum']/glb['df_'+i]['count_step']
    glb['df_'+i]['Time_avg'] = (glb['df_'+i]['Time_contract'] + glb['df_'+i]['Time_step'])/2
    glb['df_'+i].to_excel(str(i) + '.xlsx')

#### 242 группировка по срокам заключения

   ###### все договоры

In [12]:
# загружаем датасет для расчета кол-ва договоров по группам длительности
data_242 = input()
df_242 = pd.read_excel(data_242)

july_242_2020.xls


In [13]:
df_242.nunique()['ID contract']

455

In [14]:
# смотрим пропущенные данные в датасете
missingdata(df_242)

,Total,Percent
ZNP,4465,83.582928
Description,482,9.022838
Department_1,59,1.104455
Direction,40,0.748783
Profile,17,0.318233
Author,9,0.168476


In [15]:
# смотрим авторов с пропущенной дирекцией
df_242[(df_242['Direction'].isnull())].groupby(['Author']).nunique()['ID contract']

Author
Киселев Владимир Геннадьевич    2
Name: ID contract, dtype: int64

In [16]:
# заполняем пропуски по дирекции
df_242['Direction'].fillna(df_242['Author'].apply(direction), inplace = True)
missingdata(df_242)

,Total,Percent
ZNP,4465,83.582928
Description,482,9.022838
Department_1,59,1.104455
Profile,17,0.318233
Author,9,0.168476


In [17]:
# создаем столбенц с английским названием дирекции
df_242['Direction_new'] = df_242['Direction'].apply(direction_eng)

In [18]:
def group(d3):
    '''
    функция для проставление группы по длительности заключени договора
    '''
    if d3 <= 5:
        return '0-5'
    elif d3 > 5 and d3 <= 10:
        return '6-10'
    elif d3 > 10 and d3 <=15:
        return '11-15'
    elif d3 > 15 and d3 <= 30:
        return '16-30'
    elif d3 > 30 and d3 <= 60:
        return '31-60'
    else:
        return '>60'

        Внутренние этапы

In [19]:
# агрегируем по ID договора, считаем длительность заключения каждого договора только по внутренним этапам
df_242_step = df_242[df_242['Step'] != 'Работа с контрагентом'].groupby(['ID contract']).sum()['Time_days'].reset_index()
# применяем ф-цию group - проставляем группы по длительности заключения
df_242_step['Group'] = df_242_step['Time_days'].apply(group)
# считаем кол-во договоров в каждой группе
df_242_step = df_242_step.groupby(['Group']).count()['ID contract'].reset_index()
# переименовываем столбец
df_242_step.rename(columns={'ID contract': 'count_group'}, inplace=True)
# выгружаем датасет в эксель
df_242_step.to_excel('242_all_step.xlsx')

        Этап Работа с контрагентом

In [20]:
# агрегируем по ID договора, считаем длительность заключения каждого договора только по этапу Работа с к/а
df_242_rabota = df_242[df_242['Step'] == 'Работа с контрагентом'].groupby(['ID contract']).sum()['Time_days'].reset_index()
# применяем ф-цию group - проставляем группы по длительности заключения
df_242_rabota['Group'] = df_242_rabota['Time_days'].apply(group)
# считаем кол-во договоров в каждой группе
df_242_rabota = df_242_rabota.groupby(['Group']).count()['ID contract'].reset_index()
# переименовываем столбец
df_242_rabota.rename(columns={'ID contract': 'count_group'}, inplace=True)
# выгружаем датасет в эксель
df_242_rabota.to_excel('242_all_rabota.xlsx')

   ###### в разрезе дирекций

        Внутренние этапы

In [21]:
# создаем список дирекций
list_eng_242 = ['Tech',
 'PD',
 'CC']

In [22]:
# считаем аналогично кол-во договоров по группам для дирекций из списка (только внутренние этапы), создаем датасеты в эксель
for i in list_eng_242:
    glb['df_242_step_'+i] = df_242[(df_242['Direction_new'] == i) & (df_242['Step'] != 'Работа с контрагентом')].groupby(['ID contract']).sum()['Time_days'].reset_index()
    glb['df_242_step_'+i]['Group'] = glb['df_242_step_'+i]['Time_days'].apply(group)
    glb['df_242_step_'+i] = glb['df_242_step_'+i].groupby(['Group']).count()['ID contract'].reset_index()
    glb['df_242_step_'+i].rename(columns={'ID contract': 'count_group'}, inplace=True)
    glb['df_242_step_'+i].to_excel(str(i) + '_242_step.xlsx')

        Этап Работа с контрагентом

In [23]:
# считаем аналогично кол-во договоров по группам для дирекций из списка (только Работа с к/а), создаем датасеты в эксель
for i in list_eng_242:
    glb['df_242_rabota_'+i] = df_242[(df_242['Direction_new'] == i) & (df_242['Step'] == 'Работа с контрагентом')].groupby(['ID contract']).sum()['Time_days'].reset_index()
    glb['df_242_rabota_'+i]['Group'] = glb['df_242_rabota_'+i]['Time_days'].apply(group)
    glb['df_242_rabota_'+i] = glb['df_242_rabota_'+i].groupby(['Group']).count()['ID contract'].reset_index()
    glb['df_242_rabota_'+i].rename(columns={'ID contract': 'count_group'}, inplace=True)
    glb['df_242_rabota_'+i].to_excel(str(i) + '_242_rabota.xlsx')